<a href="https://colab.research.google.com/github/elio-li/CSCI4964/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Homework 4**


## **Task 1**

Data Set: <https://www.kaggle.com/mczielinski/bitcoin-historical-data>.

I would like to calculate the Bitcoin price based on features like open, high, close, etc. Sequence Models is best to solve this problem since the data is temporal information and the structure of a cyclic neural network can be useful on time sensitive data like this.

In [80]:
# Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf

In [98]:
# Read File
df = pd.read_csv("bitcoin.csv")
df.head(5)

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.0,4.39
1,1325317980,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1325318040,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1325318100,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1325318160,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
# Clean up the data
df_dropna = df.dropna()
df_dropna.head(5)

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
0,1325317920,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
478,1325346600,4.39,4.39,4.39,4.39,48.000000,210.720000,4.390000
547,1325350740,4.50,4.57,4.50,4.57,37.862297,171.380338,4.526411
548,1325350800,4.58,4.58,4.58,4.58,9.000000,41.220000,4.580000
1224,1325391360,4.58,4.58,4.58,4.58,1.502000,6.879160,4.580000


In [100]:
# Get only a small chunck of the data
df_latest = df_dropna[df_dropna['Timestamp'] > (df_dropna['Timestamp'].max()-500000)]
df_latest.head(5)

,Timestamp,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
4849043,1616648820,52411.95,52417.16,52313.93,52325.10,8.184717,428342.104410,52334.383591
4849044,1616648880,52350.85,52364.25,52318.79,52356.78,0.724648,37921.236565,52330.549717
4849045,1616648940,52356.76,52356.76,52333.24,52345.80,1.719945,90030.498364,52344.982259
4849046,1616649000,52410.90,52450.00,52410.90,52450.00,0.793599,41605.741770,52426.644103
4849047,1616649060,52450.00,52476.36,52436.61,52436.61,1.688523,88581.089260,52460.691704


In [101]:
# remove timestamp
df_choose = df_latest.drop(['Timestamp'], axis = 1)
x = np.array(df_choose.drop(['Weighted_Price'], axis = 1))
y = np.array(df_choose['Weighted_Price'])
print(x.shape)
print(y.shape)

(8306, 6)
(8306,)


In [102]:
# Scale the data
from sklearn.preprocessing import StandardScaler
x = StandardScaler().fit_transform(x)
y = (StandardScaler().fit_transform(np.reshape(y, (-1,1)))).reshape(-1)

In [103]:
x_totl = []
y_totl = []
for i in range(500, x.shape[0]) :
    x_totl.append(x[i - 500: i])
    y_totl.append(y[i])
x = np.array(x_totl)
y = np.array(y_totl)

In [104]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 1)

### **Part 1**

The RNN consists of a RNN layer as the start layer, and then all the layers still the final layer are dense layer, I used tanh as the activation function since relu produces higher loss than tanh, the final layer is linear because this is a regression problem instead of a classification problem which softmax didn't worked really well.

Matrics I used for this problem is Mean Square Error.

In [47]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import SimpleRNN

In [109]:
model = Sequential([
  SimpleRNN(10, activation='tanh', return_sequences=False),
  Dense(128, activation='tanh'),
  Dense(1, activation='linear'),
])

model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [110]:
model.fit(x_train, y_train, batch_size = 32, epochs = 10, validation_data = (x_test, y_test))

Epoch 1/10
196/196 [==============================] - 15s 75ms/step - loss: 0.0994 - val_loss: 0.0207
Epoch 2/10
196/196 [==============================] - 14s 73ms/step - loss: 0.0140 - val_loss: 0.0096
Epoch 3/10
196/196 [==============================] - 14s 72ms/step - loss: 0.0071 - val_loss: 0.0057
Epoch 4/10
196/196 [==============================] - 14s 73ms/step - loss: 0.0045 - val_loss: 0.0037
Epoch 5/10
196/196 [==============================] - 15s 75ms/step - loss: 0.0031 - val_loss: 0.0029
Epoch 6/10
196/196 [==============================] - 14s 72ms/step - loss: 0.0024 - val_loss: 0.0022
Epoch 7/10
196/196 [==============================] - 14s 73ms/step - loss: 0.0020 - val_loss: 0.0019
Epoch 8/10
196/196 [==============================] - 15s 75ms/step - loss: 0.0017 - val_loss: 0.0019
Epoch 9/10
196/196 [==============================] - 14s 73ms/step - loss: 0.0016 - val_loss: 0.0017
Epoch 10/10
196/196 [==============================] - 14s 74ms/step - loss: 0.001

In [111]:
print("Mean Square Error is: ", model.evaluate(x_test, y_test, verbose=0))

Mean Square Error is:  0.0014183245366439223


### **Part 2**

In [112]:
model = Sequential([
  LSTM(10, activation='tanh', return_sequences=False),
  Dense(128, activation='tanh'),
  Dense(1, activation='linear'),
])

model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [113]:
model.fit(x_train, y_train, batch_size = 32, epochs = 10, validation_data = (x_test, y_test))

Epoch 1/10
196/196 [==============================] - 30s 144ms/step - loss: 0.1003 - val_loss: 0.0044
Epoch 2/10
196/196 [==============================] - 28s 141ms/step - loss: 0.0026 - val_loss: 0.0018
Epoch 3/10
196/196 [==============================] - 28s 142ms/step - loss: 0.0018 - val_loss: 0.0017
Epoch 4/10
196/196 [==============================] - 28s 140ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 5/10
196/196 [==============================] - 28s 141ms/step - loss: 0.0013 - val_loss: 0.0011
Epoch 6/10
196/196 [==============================] - 28s 141ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 7/10
196/196 [==============================] - 28s 143ms/step - loss: 0.0010 - val_loss: 0.0013
Epoch 8/10
196/196 [==============================] - 28s 142ms/step - loss: 0.0010 - val_loss: 0.0010
Epoch 9/10
196/196 [==============================] - 28s 142ms/step - loss: 8.7241e-04 - val_loss: 0.0011
Epoch 10/10
196/196 [==============================] - 28s 142ms/step

We can see the mean square error looks very similar with each other on the first iterations, and after that, the loss in the second iteration in LSTM dropped significantly in terms of the rnn model, and the final the error in LSTM is significantly less than RNN.

## **Task 2**

Data Set Link: <https://tfhub.dev/google/Wiki-words-500/2>

These Resources are Used for this Task:
- <https://www.gcptutorials.com/post/sentence-similarity-using-tensorflow-pre-trained-models>
- <https://clay-atlas.com/us/blog/2020/03/27/cosine-similarity-text-calculate-python/>

In [123]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/Wiki-words-500/2")

In [156]:
word_1 = str(input())
word_2 = str(input())

vec_a, vec_b = embed([word_1])[0].numpy(), embed([word_2])[0].numpy()

dot = sum(a*b for a, b in zip(vec_a, vec_b))
norm_a = sum(a*a for a in vec_a) ** 0.5
norm_b = sum(b*b for b in vec_b) ** 0.5

similarity = dot / (norm_a*norm_b)
print("Similarit\y: ", similarity)
print("Dissimilarity: ", 1 - similarity)

dog
pet
Similarity:  0.7941820975286606
Dissimilarity:  0.20581790247133935


I've tried to find for a dissimilarity measure but sofar I haven't found a really good one, I tried to use [Bray-Curtis dissimilarity](http://www.econ.upf.edu/~michael/stanford/maeb5.pdf) but since there're negative values in the data so I can't calculate the sum of the vector. I've also found [unalikeability coefficient](https://www.amstat.org//publications/jse/v15n2/kader.html) but seems like the algorithm only works on the vector itself.

Thus I'll simply use 1 - cosine similarity as the dissimilarity because it make sense since the oposite of similarity is the dissimilarity so the dissimilarity can be the inverse of cosine similarity.

- Results

| Word A | Word B | Cosine Similarity | Cosine Dissimilarity |
| ------ | ------ | ----------------- | -------------------- |
| Human | Computer | 0.2207 | 0.7793 |
| Alpha | Beta | 0.9348 | 0.0652 |
| Mom | Dad | 0.7448 | 0.2552 |
| Apple | Fruit | 0.2995 | 0.7005 |